In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import TopKCategoricalAccuracy, CategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset = train_datagen.flow_from_directory(
    r"D:\code\archive\images2\images\train",
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle=True,
    seed=99
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_dataset = val_datagen.flow_from_directory(
    r"D:\code\archive\images2\images\validation",
    target_size=(48, 48),
    batch_size=16,
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle=True,
    seed=99
)

Found 21077 images belonging to 4 classes.
Found 5140 images belonging to 4 classes.


In [10]:
# Load ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.1)(x)  # Add dropout layer
output = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Fine-tune some layers of ResNet50
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compile the model
loss_function = CategoricalCrossentropy()
metrics = [CategoricalAccuracy(name="accuracy"), TopKCategoricalAccuracy(k=4, name="top_k_accuracy")]
model.compile(
    optimizer=Adam(learning_rate=1e-4),#1e-4 77% 47%
    loss=loss_function,
    metrics=metrics,
)

In [11]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=2e-9)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=120,
    verbose=1,
    # callbacks=[early_stopping, reduce_lr, lr_scheduler]
    callbacks=[reduce_lr, lr_scheduler]
)

Epoch 1/120


659/659 [==============================] - 1392s 2s/step - loss: 1.3590 - accuracy: 0.3636 - top_k_accuracy: 1.0000 - val_loss: 2.1731 - val_accuracy: 0.3364 - val_top_k_accuracy: 1.0000
Epoch 2/120
659/659 [==============================] - 1496s 2s/step - loss: 1.1105 - accuracy: 0.5200 - top_k_accuracy: 1.0000 - val_loss: 0.9610 - val_accuracy: 0.6072 - val_top_k_accuracy: 1.0000
Epoch 3/120
659/659 [==============================] - 633s 959ms/step - loss: 0.9806 - accuracy: 0.5891 - top_k_accuracy: 1.0000 - val_loss: 0.8699 - val_accuracy: 0.6420 - val_top_k_accuracy: 1.0000
Epoch 4/120
659/659 [==============================] - 583s 884ms/step - loss: 0.9215 - accuracy: 0.6217 - top_k_accuracy: 1.0000 - val_loss: 0.8309 - val_accuracy: 0.6626 - val_top_k_accuracy: 1.0000
Epoch 5/120
659/659 [==============================] - 584s 886ms/step - loss: 0.8654 - accuracy: 0.6439 - top_k_accuracy: 1.0000 - val_loss: 0.8190 - val_accuracy: 0.6568 - val_top_k_accuracy: 1.0000
Epoch 6/120

In [12]:
model.save('72_accuracy_model_resnet.h5')

In [14]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('72_accuracy_model_resnet.h5')


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = r"D:\code\archive\images2\images\validation\neutral\112.jpg"
img = image.load_img(img_path, target_size=(48, 48))  # Resize to (48, 48)

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Normalize the pixel values (VGG16 was trained with values ranging from 0 to 255, so we normalize by dividing by 255)
img_array /= 255.0

# Expand dimensions to match the input shape required by the model (1, 48, 48, 3)
img_array = np.expand_dims(img_array, axis=0)

In [23]:
import numpy as np

# Emotion labels used in the model
emotion_labels = ["angry", "happy", "neutral", "sad"]

# Predict the emotion
predictions = model.predict(img_array)

# Show confidence levels for each emotion
for i, emotion in enumerate(emotion_labels):
    confidence = predictions[0][i] * 100  # Convert to percentage
    print(f'{emotion}: {confidence:.2f}%')

# Get the index of the highest probability (most confident prediction)
predicted_emotion_index = np.argmax(predictions)
predicted_emotion = emotion_labels[predicted_emotion_index]

# Show the predicted emotion and its confidence level
predicted_confidence = predictions[0][predicted_emotion_index] * 100
print(f'\nPredicted emotion: {predicted_emotion} ({predicted_confidence:.2f}%)')


angry: 43.18%
happy: 0.31%
neutral: 47.46%
sad: 9.04%

Predicted emotion: neutral (47.46%)
